# Multi-scale and statistical analysis of observed and simulated astrophysical data

Jean-François ROBITAILLE, Benoît COMMERÇON & Frédérique MOTTE

Many thanks to Erik Rosolowsky for his help on *TurbuStat* functions.

This notebook presents a non-exhaustive demonstration of some python tools available to the astrophysical community to perform multi-scale and related analysis.

The first part of the hands-on project is based on mainly two packages:

 - TurbuStat - [https://github.com/Astroua/TurbuStat](https://github.com/Astroua/TurbuStat)
 - Pywavan - [https://github.com/jfrob27/pywavan](https://github.com/jfrob27/pywavan)

*TurbuStat* aimes at facilitating comparisons between spectral line data cubes. Included in this package are several techniques described in the literature which aim to describe some property of a data cube. It also includes an impressive collection of statistical analysis techniques, such as the **PDF**, the **power spectrum**, the **∆-variance**, the **dendrogram**, etc.

*Pywavan* has been developed by me and it revolves mainly around one function which is dedicated to the wavelet power spectrum analysis of a map and its **Multi-scale non-Gaussian Segmentation (MnGSeg)**. It also contains functions to perform the **classical Fourier power spectrum analysis**, generate **fractal simulations** and do basic data manipulation, such as **cutting fits maps**, **beam convolution**.

The second part of the hands-on project will explore **3D simulations dataset**. It will consist in analysing 3D simulations datasets obtained using the [RAMSES code](https://bitbucket.org/rteyssie/ramses/src/master/), from which the 2D column density maps were derived. The visualisation code [Osyris](https://github.com/nvaytet/osyris) will be used to load the 3D RAMSES datasets, to plot various quantities (density slices, histograms), to export arrays in order to do **PDF of 3D fields** for instance or to produce **column density maps**.  If time permits, the RAMSES code will be run at low resolution in order to produce directly the 3D simulation datasets, exploring the initial parameter space.

# Multi-scale & Statistical analysis

## Why multi-scale analysis techniques?

The structure formation in the interstellar medium is largely dominated by mutli-scale processes such as turbulence through its energy cascade and gravity, which is responsible for the formation of the densest structures.

Measuring the scaling behaviour of a cloud and decomposing its building blocks using several statistical techniques can help us to understand the physical mecanisms at the origin of cloud structure formation and ultimately the formation of stars.

### Dependencies

Plots and matrices (normally already installed on Colab)

In [ ]:
!pip install matplotlib numpy

Dowload and manage fits data

In [ ]:
!pip install aplpy astropy

*Turbustat* package and dependencies

In [ ]:
!pip install turbustat sklearn statsmodels radio_beam

In [ ]:
!pip install git+https://github.com/dendrograms/astrodendro.git

*Pywavan* package

In [ ]:
!pip install git+https://github.com/jfrob27/pywavan.git

In [ ]:
%pylab inline

Open data

In [ ]:
from astropy.io import fits
import aplpy

### Load a set of data

During this project, you will have to compare the statistical analysis done on multiple star forming regions (real and simulated). The real star forming regions are observed by *Herschel* space observatory and are part of the [Gould Belt Survey archive](http://www.herschel.fr/cea/gouldbelt/en/Phocea/Vie_des_labos/Ast/ast_visu.php?id_ast=66).

The first region as been chosen and prepared for you. It is the Polaris molecular cloud. Polaris is a young star forming region with just a few pre-stellar cores detected [Ward-Thompson et al. 2010](https://www.aanda.org/articles/aa/full_html/2010/10/aa14618-10/aa14618-10.html).

Once you will master the statistical analysis technique suggested in this document, you will have to compare your results will the analysis of at least another region of your choice from the [Gould Belt Survey archive](http://www.herschel.fr/cea/gouldbelt/en/Phocea/Vie_des_labos/Ast/ast_visu.php?id_ast=66).

The properties derived from the different analysis could then also be compared with the statistical simulations (fBms) you manipulated and also the numerical simulations that you will manipulate later in this project.

Let's have a look at Polaris first:

In [ ]:
#HDU = fits.open('/Users/robitaij/postdoc/colloque/notebooks/polaris-250_cut.fits')
HDU = fits.open('https://ipag.osug.fr/~robitaij/polaris-250_cutB.fits')
im = HDU[0].data
header = HDU[0].header

In [ ]:
fig_all = plt.figure(1, figsize=(10,10))

fig = aplpy.FITSFigure(fits.PrimaryHDU(im,header=header),figure=fig_all)
fig.show_colorscale(cmap='cubehelix')
fig.add_colorbar()
fig.colorbar.set_axis_label_text(header['BUNIT'])
fig.set_title("Taurus column density map")
plt.tight_layout()

### Prepare the map

The Polaris map downloaded from the archive has a lot of *NaNs* at it borders and it is also too big to be analysed directly on Colab. For this matter, we suggest you to dowload the 250 $\mu$m, which is a good proxy of the column density map and which has a lower pixel resolution.

You will need to prepare the map before starting the analysis. For this, you need to load the **subfits** function, which will allows you to cut the map and updates its header in order to keep the right celectial coordinates associated to each pixels.

In [ ]:
from pywavan import subfits

In [ ]:
#To display the manual for the function
subfits?

We also suggest, if you are working on Colab, to upload the desired map on your Google drive account and to connect you gdrive to Colab following this procedure.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Then with similar adresses (see below), you should be able to cut the desired map using the pixel coordinates that you will determine. This way your map will be saved on your gdrive and can be read again (for instance, the next day) using the same output adress.

In [ ]:
input = '/content/gdrive/MyDrive/HGBS/polaris-250.fits'
output = '/content/gdrive/MyDrive/HGBS/polaris-250_cut.fits'
coords = np.array([1180,2060,2750,3130])
subfits(input,output,coords)

Now, let's start the analysis!

# Dendrograms

In general, dendrograms provide a hierarchical description of datasets, which may be used to identify clusters of similar objects or variables. This is known as hierarchical clustering. In the case of position-position-velocity (PPV) cubes, a dendrogram is a hierarchical decomposition of the emission in the cube. This decomposition was introduced by [Rosolowsky et al. 2008](https://ui.adsabs.harvard.edu/#abs/2008ApJ...679.1338R/abstract) and [Goodman et al. 2009](https://ui.adsabs.harvard.edu/#abs/2009Natur.457...63G/abstract) to calculate the multiscale properties of molecular gas in nearby clouds. The tree structure is comprised of branches and leaves. Branches are the connections, while leaves are the tips of the branches.

In [ ]:
from turbustat.statistics import Dendrogram_Stats
from astrodendro import Dendrogram
import astropy.units as u

In [ ]:
d = Dendrogram.compute(im, min_value=0.0, min_delta=10.0, min_npix=50, verbose=False)

In [ ]:
figure(figsize=(12,9))
ax = subplot(111)  
p = d.plotter()
p.plot_tree(ax, color='black')
p.plot_tree(ax, structure=[36], lw=3, colors='yellow')
p.plot_tree(ax, structure=[34], lw=3, colors='red')
p.plot_tree(ax, structure=[77], lw=3, colors='blue')
ylabel(header['BUNIT'])
xlabel('Structure')

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(1, 1, 1)
ax.imshow(im, origin='lower', interpolation='nearest', cmap='cubehelix',vmin=-1.774e+01, vmax=5.339e+01)

# Show contour for ``min_value``
p.plot_contour(ax, colors='black')

# Highlight branches
p.plot_contour(ax, structure=36, colors='yellow')
p.plot_contour(ax, structure=34, colors='red')
p.plot_contour(ax, structure=77, colors='blue')

Branches selected using the Interactive Visualization of *astrodendro*. See instruction [here](https://dendrograms.readthedocs.io/en/stable/plotting.html).

For the relationship between the number of leaves and branches in the tree versus the minimum branch length giving a power law ([Burkhart et al. 2013](https://ui.adsabs.harvard.edu/abs/2013ApJ...770..141B/abstract)).

In [ ]:
#Very long to run!!! You could try during a talk later in the week ;-)

#dend_stat = Dendrogram_Stats(im,header=header, min_deltas='auto', dendro_params=None, num_deltas=10)
#dend_stat.run(show_progress=False)

# Power spectrum

A power spectrum measure the amount of power (squared amplitude of spatial fluctuations) as a function of spatial frequencies. The most common way to estimate the power spectrum is to average the squared amplitude as a function of scales over the azimuthal directions in the Fourier space.

Fourier transform of an image:
$$\hat{f}(\mathbf{k}) = \int^{\infty}_{-\infty} f(\mathbf{x}) e^{-2\pi i\mathbf{x} \cdot \mathbf{k}}d\mathbf{k}$$

Fourier power spectrum estimation:
$$P(k) = \langle |\hat{f}(\mathbf{k})|^2 \rangle_{\theta} \propto k^{-\beta}$$

## Fractional Brownian motion (fBm)

An fBm simulation is a simple way to generate random Gaussian fluctuations as a function of scales. These mock objects mimic ''part'' of the fractal behavior of interstellar medium and can be used to excute simple tests on statistical tools (as the power spectrum).

They are generated by calculating the inverse Fourier transform of random complex-number noise in the Fourier space multiplied by a power law"

In [ ]:
from pywavan import fbm2d
from turbustat.simulator import make_extended

In [ ]:
beta = 2.7

#Pywavan
fbm = fbm2d(-beta,256,256)

#Turbustat
fbmturbu = make_extended(256, powerlaw=beta)

figure(figsize=(15,10))
subplot(1,2,1)
imshow(fbm, origin='lower')
title('Pywavan')
subplot(1,2,2)
imshow(fbmturbu)
title('Turbustat')

Since fBms are built from a set of random-phase sinusoidal functions it has the useful property of having periodic boundaries.

In [ ]:
Lfbm = np.concatenate((fbm, fbm),axis=1)

figure(figsize=(15,10))
imshow(Lfbm, origin='lower')

## Fourier Power Spectrum analysis

In [ ]:
from pywavan import powspec

In [ ]:
tab_k, spec_k = powspec(fbm, reso = 1)
tab_k, spec_kturbu = powspec(fbmturbu, reso = 1)

#Pywavan fBm Power spectrum fit
A = np.polyfit(np.log(tab_k[1:]), np.log(spec_k[1:]),deg=1)
fit = np.exp(A[1])*tab_k**A[0]
print('Pywavan fBm Power fit')
print('A, Beta = ', np.exp(A[1]), A[0])

#Turbustat fBm Power spectrum fit
A = np.polyfit(np.log(tab_k[1:]), np.log(spec_kturbu[1:]),deg=1)
fit_turbu = np.exp(A[1])*tab_k**A[0]
print('Turbustat fBm Power fit')
print('A, Beta = ', np.exp(A[1]), A[0])

In [ ]:
figure(figsize=(16,6))
subplot(1,2,1)
plot(tab_k, spec_k)
plot(tab_k, fit, linestyle = ':')
xscale('log')
yscale('log')
xlabel('k')
ylabel('P(k)')
title('Pywavan')

subplot(1,2,2)
plot(tab_k, spec_kturbu)
plot(tab_k, fit_turbu, linestyle = ':')
xscale('log')
yscale('log')
xlabel('k')
ylabel('P(k)')
title('Turbustat')

In [ ]:
from turbustat.statistics import PowerSpectrum

In [ ]:
figure(figsize=(12,9))
pspec = PowerSpectrum(fits.PrimaryHDU(fbm))
pspec.run(verbose=True, xunit=u.pix**-1, use_wavenumber=False, fit_2D_kwargs={'fix_ellip_params':True})

## Fourier power spectrum analysis on Polaris

The scaling properties of the Polaris flare region has been analysed several times ([Miville-Deschênes et al. 2010](https://ui.adsabs.harvard.edu/abs/2010A%26A...518L.104M/abstract), [Robitaille et al. 2019](https://ui.adsabs.harvard.edu/abs/2019A%26A...628A..33R/abstract), [Allys et al. 2019](https://ui.adsabs.harvard.edu/abs/2019A%26A...629A.115A/abstract)). Polaris is particularly interesting because it is a young stellar region and it has an apparent scale-free property.

Observed star forming regions does not have periodic boundaries. This means that Fourier transforms "try" to replicate the decomposed signal to infinity, which can cause important discontinuities at the edge of the field. 

To overcome this effect:

- The mean value of the field is subtracted (power spectra only measure signal fluctuations around the mean value)
- Apodization is done at the edge of the field
- Zero padding pixels is added to the map.

In [ ]:
from pywavan import apodize, padding, gauss_beam

In [ ]:
#Resolution in arcmin
reso = header['CDELT2'] * 60.

#Apodization
imzm = im - np.mean(im)
tapper = apodize(im.shape[0],im.shape[1],0.98)
imt = imzm *tapper
nsize = 1024
imr = padding(imt,nsize,nsize)

#Fourier Powre Spectrum
tab_k_im, spec_k_im = powspec(imr, reso = reso)

In [ ]:
figure(figsize=(10,10))
imshow(imr, vmin=-1.390e+01, vmax=4.622e+01, cmap='cubehelix',origin='lower')

### Plot Fourier Power Spectrum

In [ ]:
figure(figsize=(12,9))
plot(tab_k_im, spec_k_im, label='Original')
ylim([1e-1,1e7])
xscale('log')
yscale('log')
xlabel('k')
ylabel('P(k)')
title('Polaris 250 $\mu$m')
legend()

############
#Beam and noise correction for later
############
#plot(tab_k_im, (spec_k_im-noise)/spec_beamn, label='Beam corrected')
#plot(tab_k_im, fit_im, linestyle='--', label='Fit')
#plot([tab_k_im[0],tab_k_im[-1]], [noise,noise], linestyle=':', label='Noise')

############
#Over plot turbustat result
############
#plot(pspec.wavenumbers/(1.4*60.),pspec.ps1D/(im.shape[0]*im.shape[1]))

The power spectrum display a nice power law, but not as perfect as for the fractal simualtion. Here, the Herschel map power spectrum is affected by two observational limitation:

- Instrumental noise (flat power at small scales - large $k$)
- Transfer function of the telescope (the telescope beam)

$$P(k) = \Gamma(k)P_{\textrm{sky}}(k)+N(k)$$

where,

$$P_{\textrm{sky}}(k) = A_{\textrm{ISM}}k^{\gamma} + P_{0}$$

The factor $\Gamma(k)$ is the telescope transfer function, $N(k)$ is the noise level and $P_{0}$ modelled the excess of power at small scales induced by point sources and the cosmic infrared background (CIB) associated with unresolved infrared galaxies at high redshift. In order to measure the power associated with $P_{\textrm{sky}}(k)$, the original power spectrum is first subtracted by the noise level and then divided by the telescope transfer function.

**Try to model the beam transfer function and the noise on the fBm above!**
Multiplication in the Fourier space is a convolution. So the beam can be modelled by smoothing the fbm with the **imsmooth** function of pywavan. Noise has a flat power law so

*noise* $\propto$ fbm2d(0.0,nx,ny) 

In [ ]:
#Noise evaluation
noise = np.mean(spec_k_im[(tab_k_im > 3.) & (tab_k_im < 5.)])

#Beam FWHM
FWHM = 18.2 / (60.*reso)  #Beam size at 250 micron

#Write the beam size in the header for Turbustat (some data have already the beam size written in the header)
header['BMAJ'] = 18.2 / 3600.
header['BMIN'] = 18.2 / 3600.
header['BPA'] = 0.

#Simulate the beam with a Gaussian kernel and calculate its power spectrum
beam = gauss_beam(FWHM,1024,1024,FWHM=True)
tab_k_im, spec_beam = powspec(beam, reso=reso)
spec_beamn = spec_beam/spec_beam[0]  #Spectrum normalisation

In [ ]:
#Power spectrum fit
limites = np.where((tab_k_im >= tab_k_im[1]) & (tab_k_im < 1.0))

A = np.polyfit(np.log(tab_k_im[limites]), np.log((spec_k_im[limites]-noise)/spec_beamn[limites]),deg=1)
fit_im = np.exp(A[1])*tab_k_im**A[0]
print('Pywavan Polaris Power fit')
print('A, Gamma = ', np.exp(A[1]), A[0])

In [ ]:
figure(figsize=(12,9))
pspec = PowerSpectrum(im,header)
pspec.run(verbose=True, use_wavenumber=False, xunit=u.arcmin**-1, low_cut=tab_k_im[1] / u.arcmin, high_cut=1.0 / u.arcmin,
          apodize_kernel='tukey', beam_correct=True)

## Probability density distribution (PDF)

Properties of the PDF, when related to an analytical form, have been found to correlate with changes in the turbulent properties (e.g., [Kowal et al. 2007](https://ui.adsabs.harvard.edu/abs/2007ApJ...658..423K/abstract), [Federrath et al. 2010](https://ui.adsabs.harvard.edu/abs/2010A%26A...512A..81F/abstract)), gravity (e.g., [Burkhart et al. 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...808...48B/abstract), [Burkhart et al. 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...834L...1B/abstract)) and star formation activity ([Schneider et al. 2013](https://ui.adsabs.harvard.edu/abs/2013ApJ...766L..17S/abstract)).

In [ ]:
from turbustat.statistics import PDF

### fBm PDF

Try to play with the fBm power law. **What happen for a power law $<-3.0$? Can you tell it is happening?**

In [ ]:
figure(figsize = (12,8))
pdf_fbm = PDF(fbm, bins=None)  
pdf_fbm.run(verbose=True, do_fit=False)
#Turbustat fits a lognormal PDF by default (see the reason below).
#We expect from a fBm (also called Gaussian random field) with a power law greater than -3.0
#to be Gaussian.

### Polaris PDF

In [ ]:
im_mod = im-np.min(im)+1.
im_mod = im_mod / np.mean(im_mod)
# Truncate data to be < 5
im_mod = im_mod[(im_mod < 5) & (im_mod > 0.25)]

figure(figsize = (12,8))
pdf_im = PDF(im_mod, bins=None)#, normalization_type='normalize_by_mean')  
pdf_im.run(verbose=True, do_fit=True)

**It is close to a lognormal. How the litterature explains the excess power law tail for the PDF?**

Let's try to model the lognormal distribution and interprete its origin.

### Exponentiated fBm

The exponential of a Gaussian distribution is a lognormal distribution.

$$\rho_{\rm efbm}(\mathbf{x}) = \exp [\sigma_{\rho}f(\mathbf{x})]$$

The latter is often observed in star forming regions. According to [Padoan et al. (1997)](https://ui.adsabs.harvard.edu/abs/1997ApJ...474..730P/abstract), $\sigma_{\rho}$ can be related to the Mach number, $\mathcal{M}$, as

$$\sigma_{\rho} = (\ln[1+0.5^2\mathcal{M}^2])^{0.5}$$

In [ ]:
mach = 0.8
sigrho = np.log(1+0.5**2.*mach**2.)**0.5

efbm = np.exp(sigrho*fbm)

In [ ]:
figure(figsize = (12,8))
pdf_efbm = PDF(efbm, bins=None)  
pdf_efbm.run(verbose=True, do_fit=True)

### Compare the fBm vs efBm and their power spectra

In [ ]:
tab_k, spec_ke = powspec(efbm, reso = 1)

#Pywavan fBm Power spectrum fit
A = np.polyfit(np.log(tab_k[1:]), np.log(spec_ke[1:]),deg=1)
efit = np.exp(A[1])*tab_k**A[0]
print('efBm Power fit')
print('A, Beta = ', np.exp(A[1]), A[0])

In [ ]:
figure(figsize=(15,15))
subplot(2,2,1)
imshow(fbm, origin='lower')
title('fBm')
###
subplot(2,2,2)
imshow(efbm, origin='lower')
title('efBm')
###
subplot(2,2,3)
plot(tab_k, spec_ke,label='efBm')
plot(tab_k, spec_k,label='fBm')
plot(tab_k, efit, linestyle = ':',label='fit')
xscale('log')
yscale('log')
title('Power Spectrum')
legend()
###
subplot(2,2,4)
plot(pdf_efbm.bins, pdf_efbm.pdf,label='efbm')
plot(pdf_fbm.bins, pdf_fbm.pdf,label='fbm')
yscale('log')
title('PDF')
legend()

**How to explain the very different PDFs for the two models, while having similar power spectra?**

## The $\Delta$-variance

The $\Delta$-variance technique was introduced by [Stutzki et al. 1998](https://ui.adsabs.harvard.edu/abs/1998A%26A...336..697S/abstract) as a generalization of Allan-variance, a technique used to study the drift in instrumentation. They found that molecular cloud structures are well characterized by fractional Brownian motion structure, which results from a power-law power spectrum with a random phase distribution (**We would like your thoughts on this at the end of the workshop!**). The technique was extended by [Bensch et al. 2001](https://ui.adsabs.harvard.edu/abs/2001A%26A...366..636B/abstract) to account for the effects of beam smoothing and edge effects on a discretely sampled grid. The technique was extended again by [Ossenkopf at al. 2008a](https://ui.adsabs.harvard.edu/abs/2008A%26A...485..917O/abstract), where the computation using filters of different scales was moved into the Fourier domain.

A recent application of the $\Delta$-variance was presented by [Sami Dib et al. 2020](https://ui.adsabs.harvard.edu/abs/2020arXiv200708533D/abstract).

The $\Delta$-variance spectrum is defined as

$$\sigma^2_{\Delta}(L) = \frac{1}{2}\left\langle(f(x,y)*\Psi_L)^2\right\rangle_{x,y}$$

where $*$ represents the convolution operation, $\Psi_L$ the wavelet function (**which one?**) and $L$ the angular scale.

In [ ]:
from turbustat.statistics import DeltaVariance

In [ ]:
figure(figsize=(12,9))
delvar = DeltaVariance(fits.PrimaryHDU(fbm))
delvar.run(verbose=True, xunit=u.pix) 

The relation between the power spectrum index $\beta$ ($P(k) \propto k^{-\beta}$) and the $\Delta$-variance index is ([Stutzki et al. 1998](https://ui.adsabs.harvard.edu/abs/1998A%26A...336..697S/abstract))

$$\sigma^2_{\Delta} \propto L^{\beta-2}$$

It is interesting to point out the relation between the $\beta$ index and the *Hurst exponent*

$$\beta = E + 2H$$

The *Hurst exponent* is intimately linked to the notion of mono-fractal and multi-fractal geometries that we will explore later.

### $\Delta$-variance on Polaris

In [ ]:
figure(figsize=(12,9))
delvar = DeltaVariance(fits.PrimaryHDU(imr))
delvar.run(verbose=True, xlow=delvar.lags[8], xhigh=delvar.lags[16], xunit=u.pix) 

**Is it consistent with $\beta$? Could we try to divide the beam(Even if it is not usually done for ∆-variance analysis)?**

**(Later) can you compare two datasets using turbustat.statistics.DeltaVariance_Distance?**

**Is it similar to PDF_Distance or PSpec_Distance? (could be done on fBm/efBm, Polaris/other, Polaris/simu, etc)**

**What do these metrics tell us after our experiments with fBms?**

# Wavelet Power Spectrum

Similarly to the $\Delta$-variance, a wavelet power spectrum measures the scaling behaviour of a signal while taking advantage of wavelet transforms over the Fourier transform.

- The Fourier transform transform gives the complete frequency content of a signal.
- Wavelet transforms decompose an image as a function of spatial frequencies, while keeping the spatial distribution of the signal fluctuation as a function of scales.

*Pywavan* ([Robitaille et al. 2019](https://ui.adsabs.harvard.edu/abs/2019A%26A...628A..33R/abstract)) uses the continuous and complex Morlet wavelet:

$$\Psi(\mathbf{x}) = e^{i\mathbf{k}_0\mathbf{x}}e^{-|\mathbf{x}|^2/2}$$

and its rotated version introduced by [Kirby et al. (2005)](https://ui.adsabs.harvard.edu/abs/2005CG.....31..846K/abstract).

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors

na = 512
nb = 512

ko = 5.336
delta = (2.*np.sqrt(-2.*np.log(.75)))/ko
N = int(np.pi/delta)	#Number of orientation for the Morlet wavelet

#Morlet wavelet in X space (Real part)
#----------------------------------------------

#t = 0.
t = np.pi/4.
cmap = cm.coolwarm

a=60

x = np.arange(-na/2, na/2, 1.)
y = np.arange(-nb/2, nb/2, 1.)
x, y = np.meshgrid(x, y)

Mxr = np.cos( (ko/a)*( x*np.cos(t) + y*np.sin(t) ) ) * np.exp( -0.5*(x**2+y**2)/a**2)

fig = plt.figure(figsize=(24,18))
ax = fig.add_subplot(2, 2, 1, projection='3d')

norm = colors.Normalize(vmin=-0.25,vmax=1.)

ax.plot_surface(x, y, Mxr, cmap=cmap,
                 linewidth=1.0, antialiased=True, rstride=8, cstride=8, norm=norm)
#ax.plot_wireframe(x, y, Mxr, rcount=70, ccount=70, antialiased=True, linewidth=0.7)

ax.set_zlim(-1.0, 1.0)
ax.zaxis.set_major_locator(LinearLocator(5))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax.view_init(elev=40)
ax.set_xlabel('$x$', size='x-large')
ax.set_ylabel('$y$', size='x-large')

ax.set_title('Morlet wavelet (real part)',size='xx-large')

### Wavelet tranforms of Polaris

In [ ]:
from pywavan import fan_trans

In [ ]:
fan_trans?

In [ ]:
%%time
#fan_trans do the apodization and the zero padding for you.
wt, S11a, wav_k, S1a, q = fan_trans(im, reso=reso, q=0, apodize= 0.98
                                    , arrdim = np.array([nsize,nsize]), angular=False)

### *fan_trans* outputs

wt (wavelet coefficients averaged over directions)
$$\sum_{j=0}^{N_{\theta}-1} \tilde{f}(l,\mathbf{x},\theta_j)$$

S11a (wavelet power averaged over directions)
$$P^W(l,\mathbf{x})=\frac{\delta \theta}{N_{\theta}} \sum_{j=0}^{N_{\theta}-1}  |\tilde{f}(l,\mathbf{x},\theta_j)|^2$$

S1a (wavelet power spectrum)

$$P^W(l)=\frac{1}{N_{\mathbf{x}}} \sum_{\mathbf{x}} P^W(l,\mathbf{x})$$

**You can redo the wavelet transforms with _angular=True_. Display (as below) the wavelet coefficients as a function of direction.**

**You can also calculate the _angular_ wavelet power spectra form the _wt_ output (or try later when you will master the integretion of _wt_ over the different dimensions).**

In [ ]:
figure(figsize=(12,9))
plot(tab_k_im, spec_k_im, label='Fourier')
plot(wav_k, S1a, 'D', label='Wavelet')
#plot(tab_k_im, (spec_k_im-noise)/spec_beamn, label='Beam corrected')
#plot(tab_k_im, fit_im, linestyle='--', label='Fit')
#plot([tab_k_im[0],tab_k_im[-1]], [noise,noise], linestyle=':', label='Noise')
xscale('log')
yscale('log')
xlabel('k')
ylabel('P(k)')
title('Polaris 250 $\mu$m')
legend()

### Display the wavelet coefficients

**Try different scales (or different directions!)**

In [ ]:
fig_all = plt.figure(1, figsize=(15,9))
aplpy.FITSFigure(fits.PrimaryHDU(wt[9,:,:].real,header=header)
                       ,figure=fig_all, subplot=(1,2,1)).show_colorscale(cmap='coolwarm')
###
aplpy.FITSFigure(fits.PrimaryHDU(S11a[12,:,:].real,header=header)
                       ,figure=fig_all, subplot=(1,2,2)).show_colorscale(cmap='coolwarm')

**You can integrate the coeffients over a range of scales (and/or directions) to filter scales, or even reconstruct _S1a_ from the _wt_ according to the equation.**

**Is the plateau at small scales (large $k$) on the power spectrum really associated to instrumental noise?**

In [ ]:
fig_all = plt.figure(1, figsize=(10,10))
aplpy.FITSFigure(fits.PrimaryHDU(np.sum(wt[12:,:,:].real,axis=(0)),header=header)
                       ,figure=fig_all).show_colorscale(cmap='coolwarm')

Let's have a look at the distribution of power as a function of scales.

**Compare these histograms with the wavelet transforms of an fBm and an efBm**

**From these results, can you tell why an fBm and an efBm have a similar power law?** (Think about how the wavelet power spectrum, which corresponds well to the Fourier power spectrum, is calculated)

In [ ]:
scls = [4,6,7]

plt.figure(figsize=(12,9))
for scl in scls:
    nbins = np.int(nsize**2. * (wav_k[scl]*reso)**2.)
    intermit = (S11a[scl,:,:])/np.mean(S11a[scl,:,:])
    histo, edges = np.histogram(intermit,bins=nbins)
    plt.bar(edges[:-1], histo, width=np.diff(edges), align="edge",alpha=0.5
            , label= r'$k=$ '+np.str(wav_k[scl])[:5]+r' arcmin$^{-1}$')
plt.xlabel(r'$I/\langle I \rangle$')
plt.ylabel(r"Counts")

plt.legend()

## Multiscale non-Gaussian Segmentation (MnGSeg)

This iterative algorithm was initially developed as a method to determine the optimal denoising threshold among wavelet coefficients. In many cases, denoising consists in deleting the wavelet coefficients of a noisy signal below a threshold, usually at small scales only, and reconstructing the denoised signal from the remaining coefficients [Azzalini et al. 2005](https://www.sciencedirect.com/science/article/pii/S1063520304000752). In our case, the algorithm is applied at every spatial scales and as a function of the azimuthal direction.

The algorithm is defined as follows, let $\Phi$ be the threshold splitting the non-Gaussian terms from the Gaussian terms in the wavelet coefficient distribution and $\mathbb{L}_{\Phi}$ be the function indicator. The threshold $\Phi$ is first estimated according to the variance,


$$\sigma_{l,\theta}^2(\Phi)=\frac{1}{N_{l,\theta}(\Phi)}\sum_{\mathbf{x}} \mathbb{L}_{\Phi}( |\tilde{f}_{l,\theta}(\mathbf{x})|) |\tilde{f}_{l,\theta}(\mathbf{x})|^2,$$

where

$$\mathbb{L}_{\Phi}(|\tilde{f}_{l,\theta}(\mathbf{x})|)=\left\{
        									          \begin{array}{ll}
  									          1 & \qquad \mathrm{if} \quad |\tilde{f}_{l,\theta}(\mathbf{x})| < \Phi \\
 									          0 & \qquad \mathrm{else}. \\
 									         \end{array}
  									         \right.$$

and


$$N_{l,\theta}(\Phi)=\sum_{\mathbf{x}} \mathbb{L}_{\Phi}( |\tilde{f}_{l,\theta}(\mathbf{x})|).$$

The iterative calculation then converges to an optimal value of the threshold $\Phi$ which allows one to separate outliers from randomly distributed wavelet coefficients. The sequence is defined by:


$$\left\{
        	 \begin{array}{l}
  	   \Phi_{0}(l,\theta)=\infty \\
 	   \Phi_{n+1}(l,\theta)=q \sigma_{l,\theta}(\Phi_{n}(l,\theta)), \\
 	 \end{array}
  	 \right.$$

where $q$ is a dimensionless constant controlling how restrictive is the definition of non-Gaussianities.

In [ ]:
%%time
M = wav_k.size
q=[2.0]*M

wt, S11a, wav_k, S1a, q = fan_trans(im, reso=reso, q=q, qdyn=False, apodize= 0.98
                                    , arrdim = np.array([nsize,nsize]), angular=False)

In [ ]:
coherent = np.sum(wt[M:2*M,:,:],axis=0)  + np.mean(im)
Gaussian = np.sum(wt[2*M:3*M,:,:],axis=0)  + np.mean(im)

fig_all = plt.figure(1, figsize=(20,10))

fig = aplpy.FITSFigure(fits.PrimaryHDU(Gaussian.real,header=header), figure=fig_all, convention='calabretta',subplot=(1, 2, 1))
fig.show_colorscale(cmap='cubehelix')
fig.tick_labels.set_xformat('dd.d')
fig.tick_labels.set_yformat('dd.d')
fig.add_colorbar()

fig = aplpy.FITSFigure(fits.PrimaryHDU(coherent.real,header=header), figure=fig_all, convention='calabretta',subplot=(1, 2, 2))
fig.show_colorscale(cmap='cubehelix')
fig.tick_labels.set_xformat('dd.d')
fig.tick_labels.set_yformat('dd.d')
fig.add_colorbar()

In [ ]:
figure(figsize=(12,9))
plot(tab_k_im, spec_k_im, label='Fourier')
plot(wav_k, S1a[1,:], 'D', label='Coherent')
plot(wav_k, S1a[2,:], 'D', label='Gaussian')
#plot(tab_k_im, (spec_k_im-noise)/spec_beamn, label='Fourier')
#plot(wav_k, (S1a[1,:]-noise)/S1aBn, 'D', label='Coherent')
#plot(wav_k, (S1a[2,:]-noise)/S1aBn, 'D', label='Gaussian')
#plot(wav_k, fitG, '--', label='Gaussian fit')
xscale('log')
yscale('log')
xlabel('k')
ylabel('P(k)')
title('Polaris 250 $\mu$m')
legend()

Compared to [Robitaille et al. 2019](https://ui.adsabs.harvard.edu/abs/2019A%26A...628A..33R/abstract) (below), power spectra are not as different as for the segmentation of the full Polaris map. This can be explained by the fact that the power spectrum is a statistical analysis, so larger the sample is, better is the analysis. Nevertheless, you should succeed to retrieve a power law for the Gaussian component of the extraction.

**Plot the distribution of power as a function of scales for the Gaussian component only. It should be similar to power histograms of an fBm. Why?**

<img src="https://ipag.osug.fr/~robitaij/aa35545-19-fig12.png" width=500 height=500 />

### Beam correction for the wavelet spectrum

In [ ]:
wtB, S11aB, wav_k, S1aB, qB = fan_trans(beam, reso=reso, q=0, angular=False)
del wtB, S11aB, qB

In [ ]:
#Normalisation
S1aBn = S1aB/S1aB[3]
S1aBn[:3] = 1.

In [ ]:
figure(figsize=(12,9))
plot(wav_k,  S1aBn,'d', label='Fourier')
xscale('log')
yscale('log')
xlabel('k')
ylabel('P(k)')
title('Beam')

In [ ]:
#Power spectrum fit

Ag = np.polyfit(np.log(wav_k[7:17]), np.log((S1a[2,7:17]-noise)/S1aBn[7:17]),deg=1)
fitG = np.exp(Ag[1])*wav_k**Ag[0]
print('Gaussian Power fit')
print('A, Gamma = ', np.exp(Ag[1]), Ag[0])

## Comparison with fBms

In [ ]:
from pywavan import imsmooth

In [ ]:
fbm_polaris = fbm2d(Ag[0],im.shape[1],im.shape[0])

figure(figsize=(20,10))
subplot(1,2,1)
imshow(imsmooth(fbm_polaris,FWHM),origin='lower')

subplot(1,2,2)
imshow(Gaussian.real,origin='lower')

## Multiplicative random cascade model

As noted by [Vazquez-Semadeni (1994)](https://ui.adsabs.harvard.edu/abs/1994ApJ...423..681V/abstract), hierarchical structures are naturally expected to arise in flows in which the density has a log-normal distribution. This emergence of a universal distribution can be associated with the central limit theorem, for which the sum of a large number of random variables converges to a normal distribution. If this assumption is attributed to a random sequence of $\ln\rho$, then the density $\rho$ becomes log-normally distributed. Thus, according to this description, after a finite time, density can be considered the product of a large number of independent random fluctuations $\delta$,

$$\rho(t_n)=\delta_n\delta_{n-1}\textrm{... }\delta_1\delta_0\rho(t_0)$$

We note the correspondence between the equation above and the multiplicative random cascade model that is designed to reproduce the multifractal nature of intermittency in developed turbulence ([Meneveau & Sreenivasan 1991](https://ui.adsabs.harvard.edu/abs/1991JFM...224..429M/abstract); Frisch 1995, [Robitaille et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...641A.138R/abstract)). In this model, the rate of turbulent energy dissipation $\varepsilon$ is considered first as a nonrandom positive quantity for a cubic volume of side $\ell_0$. This volume is then subdivided into eight equal cubes of side $\ell_1 = \ell_0/2$. For every subdivisions, the dissipation $\varepsilon$ is multiplied by a series of independent random variables $W_n$ that are distributed identically. After $n$ generations, the dissipation value of a cube of side $\ell_n = \ell_02^{-n}$  becomes $\varepsilon_{\ell}=W_nW_{n-1}\textrm{... }W_1W_0\varepsilon$.

**If we can model the Gaussian (turbulent?) component of the signal, can we model the coherent part?**

In [ ]:
wtA, S11aA, wav_kA, S1aA, qA = fan_trans(fbm_polaris, reso=reso, q=0, angular=True)
del S11aA

### A multiplicative cascade of the wavelet coefficients

In [ ]:
mfrac = np.zeros(fbm_polaris.shape)
Ndir = wtA.shape[1]
coeff = 0.202     #Mach=1.5
mach2 = np.sqrt(2*(np.exp((coeff)**2.)-1))
print('Mach=',mach2)

for j in range(Ndir):
    casc=np.zeros(fbm_polaris.shape)   
    for i in range(np.size(wav_kA)):
        aa=np.copy(wtA[i,j,:,:].real)
        casc+=aa/np.std(aa)*coeff
    mfrac+=np.exp(casc)
    
mfrac = mfrac / Ndir

In [ ]:
fig_all = plt.figure(1, figsize=(15,9))
aplpy.FITSFigure(fits.PrimaryHDU(imsmooth(mfrac,FWHM),header=header)
                       ,figure=fig_all, subplot=(1,2,1)).show_colorscale(cmap='cubehelix')
###
aplpy.FITSFigure(fits.PrimaryHDU(coherent.real,header=header)
                       ,figure=fig_all, subplot=(1,2,2)).show_colorscale(cmap='cubehelix')

A little similar... but if we look at the basic statistics.

In [ ]:
mfrac_mod = mfrac / np.mean(mfrac)
# Truncate data to be < 5
mfrac_mod = mfrac_mod[(mfrac_mod < 20)]

figure(figsize = (12,8))
pdf_mfrac = PDF(mfrac_mod, bins=None)  
pdf_mfrac.run(verbose=True, do_fit=True)

In [ ]:
tab_k_mfrac, spec_k_mfrac = powspec(mfrac, reso = reso)

figure(figsize=(12,9))
plot(tab_k_mfrac, spec_k_mfrac)
xscale('log')
yscale('log')
xlabel('k')
ylabel('P(k)')

**Can we relate these random multiplicative cascade structures to the statistical properties of dense ISM structures?**

# Numerical simulations

Let's have a look at numerical simulations corresponding to simulations k=2, Dcr=1d26 of [Commercon et al. (2019)](https://ipag.osug.fr/~robitaij/2019_CRMIS.pdf).

They are column density maps (g/cm$^2$) showing the CNM/WNM transition.

In [ ]:
cold1 = np.loadtxt("https://ipag.osug.fr/~robitaij/cold1_Dcr26_k2.txt")
cold2 = np.loadtxt("https://ipag.osug.fr/~robitaij/cold2_Dcr26_k2.txt")
cold3 = np.loadtxt("https://ipag.osug.fr/~robitaij/cold3_Dcr26_k2.txt")

In [ ]:
figure(figsize=(30,10))

subplot(1,3,1)
imshow(cold1)
axis('off')
title('cold1_Dcr26_k2')

subplot(1,3,2)
imshow(cold2)
axis('off')
title('cold2_Dcr26_k2')

subplot(1,3,3)
imshow(cold3)
axis('off')
title('cold3_Dcr26_k2')

tight_layout()

In [ ]:
Lcold2 = np.concatenate((cold2, cold2),axis=1)

figure(figsize=(15,10))
imshow(Lcold2, origin='lower')

### Now your turn!